20221010 

In [13]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilution, replenish, dilute_out_species
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [35]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =1
T7_DNA.lc =1

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 1
GFP_DNA.lc = 1

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0.2
Repressor_DNA.lc = 0.2


#resources
R.ic =10
R.lc =10

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):

    molecules_0[i] = molecules_list[i].ic


In [15]:
#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [16]:
#stage 1 kick start
result_all=molecules_0 #initial condition

DR = 0.2
Switch_time = np.array([4,16,24])
Switch_cycle = Switch_time*4

dilution_list = [T7_RNA,GFP,GFP_RNA,Repressor,Repressor_RNA]
replenish_list = [T7,R]

for n in range (1,Switch_cycle[0]):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Repressor_model, molecules_0, t, args=parameters_list)
    result_all = np.append(result_all,result[1])
    #update parameter
    molecules_0 = result.transpose()[:,-1]
     
    #dilution & replenish

    ###diute out
    dilute_out_species((T7_RNA,GFP,GFP_RNA,Repressor,Repressor_RNA),molecules_0)

    ###replenish 
    #replenish((R,T7),molecules_0)
    molecules_0[R.idx] = R.ic*DR+(1-DR)*molecules_0[R.idx]
    molecules_0[T7.idx] = T7.lc*DR+(1-DR)*molecules_0[T7.idx]



#stage 2 self regeneration 

for n in range (Switch_cycle[0],Switch_cycle[1]):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Repressor_model, molecules_0, t, args=parameters_list)
    result_all = np.append(result_all,result[1])
    
    #update parameter
    molecules_0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules_0[R.idx] = R.ic*DR+(1-DR)*molecules_0[R.idx]
    

    ###diute out
    dilute_out_species((T7_RNA,GFP,GFP_RNA,Repressor,Repressor_RNA),molecules_0)



for n in range (Switch_cycle[1],Switch_cycle[2]):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Repressor_model, molecules_0, t, args=parameters_list)
    result_all = np.append(result_all,result[1])
    #update parameter
    molecules_0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules_0[R.idx] = R.ic*DR+(1-DR)*molecules_0[R.idx]

    ###diute out
    dilute_out_species((T7,T7_RNA,T7_DNA,GFP,GFP_RNA,GFP_DNA,Repressor,Repressor_RNA,Repressor_DNA),molecules_0)



result_all = np.resize(result_all,(96,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()


Unexpected exception formatting exception. Falling back to standard exception


TypeError: only size-1 arrays can be converted to Python scalars

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/william/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/wg/xccqvpcs6951v9_ztz2fp4180000gn/T/ipykernel_10934/1864559938.py", line 27, in <cell line: 11>
    dilute_out_species((T7_RNA,GFP,GFP_RNA,Repressor,Repressor_RNA),molecules_0)
  File "/Users/william/Library/Mobile Documents/com~apple~CloudDocs/Desktop/GRN_Chemostat/GRN-in-chemostat/scripts/functions.py", line 15, in dilute_out_species
ValueError: setting an array element with a sequence.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/william/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1982, in showtraceback
    st

In [20]:
Switch_time = np.array([4,16,24])
Switch_cycle = Switch_time*4
print(Switch_cycle)

[16 64 96]


In [21]:
t = np.linspace(0, 1440, 96)
p = bokeh.plotting.figure(
    plot_width=800,
    plot_height=400,
    x_axis_label="t",
    y_axis_type="linear",

)

colors = bokeh.palettes.d3["Category10"][3]


# Populate glyphs
p.line(
    t/60, T7.result, line_width=2, color=colors[0]
)
vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
p.add_layout(vline1)
p.add_layout(vline2)


show(p)


In [32]:
def dilute(molecule_diluted,molecules_0,DR=0.2):
    
    molecules_0[molecule_diluted.idx] *= (1-DR)

def replenish(molecule_replenished, molecules_0, DR=0.2) : 

    molecules_0[molecule_replenished.idx] += molecule_replenished.lc * DR


def dilute_out_species(molecules_diluted,molecules_0,DR=0.2):  #dilute a list of molecules
    for molecule in (molecules_diluted):
        dilute(molecule,molecules_0,DR)

def replenish_species(molecules_replenished, molecules_0, DR=0.2) :   #replenish a list of molecules

   for molecule in (molecules_replenished):
        replenish(molecule,molecules_0,DR)


dilute_out_species((T7,T7_DNA),molecules_0)
print(molecules_0)

[2.14748365e-02 7.77218037e-01 5.56172462e-02 1.00000000e+00
 5.71956445e+00 1.43999338e+00 2.00000000e-01 1.14391289e+00
 2.87998677e-01 2.79359728e-05]


In [48]:
molecules_0=[1,1,1,1,1,1,1,1,1,1]
for i in range(10):
    dilute_out_species((T7,T7_DNA),molecules_0)
    print(molecules_0)
    T7.lc = 0.1
    replenish_species((T7,T7_DNA),molecules_0)

    print(molecules_0)
print(molecules_list)
for molecule in molecules_list:
    print(molecule.n)

[0.8, 1, 0.8, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.8200000000000001, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.6560000000000001, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.6760000000000002, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.5408000000000002, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.5608000000000002, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.44864000000000015, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.46864000000000017, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.37491200000000013, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.39491200000000015, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.31592960000000014, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.33592960000000016, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.26874368000000015, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.28874368000000017, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.23099494400000015, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.25099494400000016, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.20079595520000015, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.22079595520000017, 1, 1, 1, 1, 1, 1, 1]
[0.8, 1, 0.17663676416000015, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1, 0.19663676416000014, 1, 1, 1, 1, 1, 1, 1]
[

AttributeError: 'DNA' object has no attribute '__name__'